# 🧠 Programación Dinámica (PD): qué es y cuándo usarla

**Idea central:** resolver un problema grande descomponiéndolo en **subproblemas** cuyos resultados **se reutilizan**.
Dos propiedades clave:

* **Subestructura óptima** ⚙️: la solución óptima del problema se compone de soluciones óptimas de subproblemas.
* **Subproblemas superpuestos** 🔁: el árbol de recursión repite los mismos subproblemas una y otra vez.

**Memoization (top-down)** = *recursión + caché*. Guardas el resultado de cada subproblema la primera vez que lo calculas y, si vuelve a aparecer, lo lees de la caché en O(1).

> **Regla de oro de complejidad en PD**:
> **Tiempo ≈ #estados distintos × costo por estado**
> **Espacio ≈ #estados almacenados + profundidad del call stack**

---

# 🧭 Memoization vs. Tabulation

* **Memoization (Top-Down)** 🧗‍♂️

  * Empiezas por el problema completo y desciendes recursivamente.
  * Calculas *sólo* los estados que realmente se necesitan.
  * Más natural cuando ya tienes una recursión clara.
  * Cuidado con la **profundidad de pila** y **mutabilidad de claves**.

* **Tabulation (Bottom-Up)** 🧱

  * Llenas una tabla iterativamente desde casos base.
  * Evitas recursión y controlas mejor memoria/orden.
  * Requiere pensar el **orden de llenado**.

> En clase: empieza con **memoization** (más intuitivo), y luego muestra cómo convertirlo a **tabulation**.

---

# 🧩 Anatomía de una solución con memoization

1. **Define el estado** 🎯: ¿qué parámetros hacen único al subproblema? (índices, capacidad, posición, etc.)
2. **Escribe la recursión “de fuerza bruta”** ✍️: sin preocuparte por eficiencia (claro y correcto).
3. **Casos base** 🪜: condiciones de frontera que cortan la recursión.
4. **Transición** 🔀: cómo combinas subsoluciones (min, max, suma, conteo, etc.).
5. **Caché** 🗃️: diccionario o `@lru_cache`. Claves **inmutables** (tuplas, enteros, strings).
6. **Complejidad** 📏: cuenta estados (tamaño del dominio del estado) y el costo por estado.
7. **Validación** ✅: prueba con ejemplos pequeños; activa/inspecciona la caché si dudas.

---

# 🧠 Modelo mental (checklist de diseño)

* 🔎 **¿Hay recomputación?** Dibuja un árbol de recursión pequeño y busca subllamadas repetidas.
* 🧱 **Estado mínimo**: “si dos llamadas tienen el mismo conjunto de hechos relevantes, deben compartir estado”.
* ⛳ **Base cases claros**: que no dependan de la caché; evita ciclos.
* 🧮 **Transición limpia**: que cada camino de decisión reduzca el problema (convergencia).
* 🧭 **Clave de caché**: usa tuplas de parámetros inmutables; **no** metas listas/objetos mutables.
* 🧯 **Efectos laterales**: memoization es para funciones *puras* (mismo input → mismo output).
* 📦 **Memoria**: ¿cuántos estados caben? Si el dominio es grande, evalúa *tabulation* con compresión de estados.

---

# 🧪 Plantilla universal (Python, con y sin `lru_cache`)

### Con `functools.lru_cache`

```python
from functools import lru_cache

@lru_cache(maxsize=None)
def dp(estado1: int, estado2: int) -> int:
    # 1) casos base
    if condicion_de_base:
        return valor_base
    # 2) transiciones (subllamadas)
    opcionA = dp(nuevo_estadoA1, nuevo_estadoA2)
    opcionB = dp(nuevo_estadoB1, nuevo_estadoB2)
    # 3) combinación (min/max/suma)
    return min(opcionA, opcionB)  # o max / suma / etc.
```

### Con diccionario manual

```python
from typing import Dict, Tuple

def solve(param1: int, param2: int) -> int:
    memo: Dict[Tuple[int, int], int] = {}

    def dp(a: int, b: int) -> int:
        # caché
        if (a, b) in memo:
            return memo[(a, b)]

        # casos base
        if condicion_de_base:
            memo[(a, b)] = valor_base
            return valor_base

        # transiciones
        res = ... # combina dp(subestado...) según la recurrencia
        memo[(a, b)] = res
        return res

    return dp(param1, param2)
```

👉 **Explicación rápida:**

* Elegimos **clave** como tupla de parámetros del subproblema.
* Guardamos el resultado antes de retornar.
* Así, **cada estado se calcula una sola vez**.

---

# 🎓 Ejemplos guiados

## 1) Fibonacci (ilustra superposición de subproblemas)

**Estado:** `f(n)`
**Base:** `f(0)=0`, `f(1)=1`
**Transición:** `f(n)=f(n-1)+f(n-2)`
**Complejidad:** tiempo O(n), espacio O(n) con memo (O(n) caché + O(n) call stack)

```python
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n: int) -> int:
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)
```

**Por qué funciona:** sin memo hay 2^n llamadas; con memo cada `n` se calcula una vez.

---

## 2) LCS – Longest Common Subsequence (longitud)

**Problema:** longitud de la subsecuencia común más larga entre `s` y `t`.
**Estado:** `(i, j)` = longitud de LCS entre `s[i:]` y `t[j:]`.
**Base:** si `i==len(s)` o `j==len(t)`, LCS = 0.
**Transición:**

* Si `s[i]==t[j]`: `1 + LCS(i+1, j+1)`
* Si no: `max(LCS(i+1, j), LCS(i, j+1))`
  **#estados:** `O(len(s)*len(t))`.
  **Complejidad:** tiempo O(n·m), espacio O(n·m) (caché) + O(n+m) (pila).

```python
from functools import lru_cache

def lcs_len(s: str, t: str) -> int:
    n, m = len(s), len(t)

    @lru_cache(maxsize=None)
    def dp(i: int, j: int) -> int:
        if i == n or j == m:
            return 0
        if s[i] == t[j]:
            return 1 + dp(i+1, j+1)
        return max(dp(i+1, j), dp(i, j+1))

    return dp(0, 0)
```

**Claves didácticas:** muestra cómo el árbol de recursión se “aplana” a una malla `i×j`.

---

## 3) Caminos en una grilla con obstáculos (conteo)

**Problema:** dado un grid `grid` de 0/1, contar caminos de `(0,0)` a `(n-1,m-1)` moviendo solo derecha/abajo sin pisar celdas con `1`.
**Estado:** `(i, j)` = #caminos desde `(i, j)` a la meta.
**Base:** si `(i, j)` es la meta → 1; si está fuera o hay obstáculo → 0.
**Transición:** `dp(i, j) = dp(i+1, j) + dp(i, j+1)`
**#estados:** `O(n·m)`
**Complejidad:** tiempo O(n·m), espacio O(n·m) + O(n+m).

```python
from functools import lru_cache
from typing import List

def count_paths(grid: List[List[int]]) -> int:
    n, m = len(grid), len(grid[0])

    @lru_cache(maxsize=None)
    def dp(i: int, j: int) -> int:
        if i >= n or j >= m or grid[i][j] == 1:
            return 0
        if i == n-1 and j == m-1:
            return 1
        return dp(i+1, j) + dp(i, j+1)

    return dp(0, 0)
```

**Observación:** si cambias la suma por `min`/`max` + costos, obtienes la variante de **camino mínimo**.

---

# 🚩 Errores comunes y cómo evitarlos

* **Claves mutables en la caché** ❌ → usa **tuplas**/enteros/strings.
* **Olvidar casos base** → bucles infinitos o recursión profunda.
* **Estados incompletos** → colisiones en la caché (resultados incorrectos).
* **Side effects** en funciones memoizadas → resultados incoherentes.
* **Reinicializar la caché** en cada llamada externa inadvertidamente.
* **Profundidad de recursión** en inputs grandes → evaluar *tabulation* o `sys.setrecursionlimit` con cuidado.

---

# 🧮 Cómo analizar complejidad (plantilla)

1. **Cuenta estados**: producto/cartesiano de los rangos de cada parámetro del estado.

   * Ej.: LCS → `|i|∈[0..n]`, `|j|∈[0..m]` ⇒ `O(n·m)` estados.
2. **Costo por estado**: cuántas transiciones evalúas y su costo.

   * Suele ser O(1)–O(grado de opciones).
3. **Tiempo total**: `#estados × costo_por_estado`.
4. **Espacio**: memoria de caché + call stack (profundidad máxima).

---

# 🛠️ Convertir a Bottom-Up (cuando conviene)

* Define **orden topológico** natural (por tamaños crecientes, índices decrecientes, etc.).
* **Reusa memoria** (rolling arrays) si la transición solo usa filas/columnas previas.
* Evita **desbordes de pila** y mejora **localidad de caché** de CPU.

*Ej.: LCS bottom-up llena una tabla `n+1 × m+1` desde el fondo; se puede comprimir a 2 filas si solo lees la fila siguiente.*

---

# 📚 Mini-banco de ejercicios intro (perfectos para memoization)

1. **Stairs / Climbing**: #formas de subir `n` escalones con pasos 1 o 2.
2. **Decode Ways**: cuántas decodificaciones para un string de dígitos.
3. **Coin Change** (conteo y/o mínimo #monedas).
4. **LCS / Edit Distance** (par de strings).
5. **Grid Paths / Min Path Sum** con obstáculos.

---


## Ejemplo: Fibonacci

In [ ]:
#fuerza bruta
def fib(n: int) -> int:
  if(n < 2):
    return n

  return fib(n-1) + fib(n-2)

fib(100)

KeyboardInterrupt: 

---

## 🔧 Código con diccionario manual

```python
from typing import Dict

def fib(n: int) -> int:
    # caché inicial con los casos base
    memo: Dict[int, int] = {0: 0, 1: 1}

    def dp(k: int) -> int:
        # si ya está en el diccionario, lo devolvemos en O(1)
        if k in memo:
            return memo[k]
        
        # si no está, lo calculamos recursivamente
        memo[k] = dp(k-1) + dp(k-2)
        return memo[k]

    return dp(n)


# Ejemplo de uso
print(fib(10))  # 55
```

---

## 🧠 Explicación paso a paso

1. **Definimos la caché** como un diccionario `memo` donde la **clave** es el índice `n` y el **valor** es `fib(n)`.
2. **Inicializamos** con los **casos base** `{0:0, 1:1}`.
3. En la función recursiva `dp(k)`:

   * Primero verificamos si `k` ya está en `memo`.

     * ✅ Sí → devolvemos el valor en O(1).
     * ❌ No → lo calculamos recursivamente y lo guardamos.
4. Al final, siempre que se vuelva a necesitar `fib(k)`, se reutiliza desde `memo`.

---

## 📊 Complejidad

* **Tiempo:** O(n), porque cada valor de `fib(k)` se calcula una sola vez.
* **Espacio:**

  * **Memo:** O(n) (almacena todos los resultados hasta `fib(n)`).
  * **Call stack:** O(n) en la versión recursiva (puede optimizarse si se pasa a versión iterativa).

---

## 🧪 Ejemplo de trazado

Si llamamos `fib(5)`:

```
fib(5) = fib(4) + fib(3)
fib(4) = fib(3) + fib(2)
fib(3) = fib(2) + fib(1)
fib(2) = fib(1) + fib(0)
```

* Se calculan en orden hasta `fib(0)` y `fib(1)`.
* Luego todos los resultados quedan **guardados en memo**.
* Así, si luego pedimos `fib(6)`, ya no recalcula desde cero: aprovecha el diccionario.


In [ ]:
from typing import Dict

def fib(n: int) -> int:
    # caché inicial con los casos base
    memo: Dict[int, int] = {0: 0, 1: 1}

    def dp(k: int) -> int:
        # si ya está en el diccionario, lo devolvemos en O(1)
        if k in memo:
            return memo[k]

        # si no está, lo calculamos recursivamente
        memo[k] = dp(k-1) + dp(k-2)
        return memo[k]

    return dp(n)



fib(100)

354224848179261915075

In [ ]:
#enfoque lru_cache
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n: int) -> int:
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

fib(100)
fib.cache_info()

CacheInfo(hits=98, misses=101, maxsize=None, currsize=101)

---

# 🗂️ Guía Básica de Hashmaps

## 1. 🌐 ¿Qué es un Hashmap?

Un **hashmap** (o **tabla hash**) es una estructura de datos que guarda **pares clave → valor** y permite:

* **Insertar** un valor asociado a una clave.
* **Buscar** un valor a partir de su clave.
* **Eliminar** un par clave-valor.

### ⚡ Ventaja principal:

El acceso a los datos es, en promedio, **O(1)** (tiempo constante).
👉 Es decir: no importa si tienes 10 o 1 millón de elementos, la búsqueda tarda lo mismo.

---

## 2. 🔑 El concepto de **hashing**

1. **Clave original**: puede ser un número, string, tupla, etc.
2. **Función hash**: transforma esa clave en un número (índice).

   * Ejemplo: `"perro"` → `hash("perro") = 248163`
3. **Índice en arreglo**: ese número se usa para decidir dónde guardar el valor.

```text
clave → hash(clave) → índice → valor
```

### Problema típico: **colisiones**

* Dos claves distintas pueden producir el mismo índice.
* Estrategias:

  * **Encadenamiento**: guardar varios elementos en la misma posición como lista.
  * **Open addressing**: buscar la siguiente posición libre.

---

## 3. 📖 Hashmaps en Python → `dict`

En Python, el `dict` **es un hashmap optimizado**.

* Claves: deben ser **inmutables y hashables** (`int`, `str`, `tuple`, `frozenset`, …).
* Valores: cualquier objeto.
* Operaciones (`O(1)` promedio):

  * `d[k] = v` → inserta.
  * `d[k]` → busca.
  * `k in d` → verifica existencia.

Ejemplo:

```python
edades = {"Ana": 21, "Juan": 25, "Luis": 30}

print(edades["Ana"])    # 21
edades["Juan"] = 26     # actualizar
print("Luis" in edades) # True
```

---

## 4. 🧠 Relación con **Programación Dinámica**

### Memoization = Hashmap de subproblemas

Cuando resolvemos problemas con PD:

* **Clave** = el estado del subproblema (ejemplo: `n` en Fibonacci, `(i,j)` en LCS).
* **Valor** = el resultado calculado de ese estado.

Ejemplo con Fibonacci:

```python
def fib(n: int, memo=None) -> int:
    if memo is None:
        memo = {0: 0, 1: 1}
    if n in memo:
        return memo[n]
    memo[n] = fib(n-1, memo) + fib(n-2, memo)
    return memo[n]

print(fib(10))  # 55
```

### ¿Qué está pasando?

1. Cuando llamo `fib(10)`, la clave es `10`.
2. Si ya está en `memo`, devuelve en O(1).
3. Si no, lo calcula, lo guarda y queda listo para reutilizarse.

---

## 5. 🎓 Modelo mental para estudiantes

* Piensa en un `dict` como una **libreta de apuntes** 📒:

  * La **clave** es el título de la nota (`"fib(10)"`).
  * El **valor** es el contenido ya resuelto (`55`).
* Cada vez que vuelves a necesitarlo, **no resuelves el problema otra vez**, solo **miras en tu libreta**.
* Así funciona **memoization** en PD → un **hashmap que evita cálculos repetidos**.

---

## 6. 🚩 Ejemplo comparativo

### Sin memoization (fuerza bruta)

```python
def fib(n: int) -> int:
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print(fib(35))  # muy lento 😓
```

### Con diccionario (hashmap)

```python
def fib(n: int, memo=None) -> int:
    if memo is None:
        memo = {0: 0, 1: 1}
    if n in memo:   # lookup O(1)
        return memo[n]
    memo[n] = fib(n-1, memo) + fib(n-2, memo)
    return memo[n]

print(fib(35))  # rápido ⚡
```

---

# ✅ Resumen final

* **Hashing** = transformar una clave en un número (índice).
* **Hashmap (dict en Python)** = estructura que guarda clave→valor con acceso O(1).
* **Memoization** = usar un `dict` para **almacenar resultados de subproblemas** y evitar recalcular.
* **Beneficio** en PD: cada estado se calcula **una sola vez**.

---

# 🧠 `functools.lru_cache` a fondo (memoization en Python)

`lru_cache` es un **decorador** que añade **memoization** a una función: guarda resultados de invocaciones previas y, si vuelves a llamar con los **mismos argumentos**, devuelve el resultado desde la caché en **O(1)** amortizado. Además, mantiene una política **LRU (Least Recently Used)** para decidir **qué expulsar** cuando la caché llega a su límite.

---

## 🔧 Uso básico

```python
from functools import lru_cache

@lru_cache(maxsize=None)  # None = caché sin límite (memoization “pura”)
def fib(n: int) -> int:
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)
```

* `@lru_cache(...)` envuelve la función con una capa de caché.
* La **clave** de caché se construye con los **args/kwargs** (deben ser **hashables**).
* Con `maxsize=None` nunca se expulsa nada (útil en DP top-down sobre dominios acotados).
* Con `maxsize` finito (p. ej. 1024) aplica política **LRU**: expulsa lo menos usado recientemente.

---

## ⚙️ Parámetros importantes

* **`maxsize`**

  * `None`: caché ilimitada (memoization pura).
  * `0`: **desactiva** la caché (deja el decorador pero sin almacenamiento).
  * `N > 0`: tamaño máximo; cuando se llena, expulsa con LRU.
* **`typed`** (`False` por defecto)

  * Si `True`, **distingue tipos** en la clave: `1` y `1.0` se tratan como diferentes, `True` y `1` también.
  * Útil cuando la función depende sutilmente del tipo (no solo del valor).

> 🧩 Desde Python 3.9 existe `functools.cache`, que es equivalente a `lru_cache(maxsize=None)`.

---

## 🧪 Inspección y mantenimiento

Toda función decorada expone dos utilidades:

```python
f.cache_info()   # → CacheInfo(hits=..., misses=..., maxsize=..., currsize=...)
f.cache_clear()  # limpia completamente la caché
```

Ejemplo:

```python
@lru_cache(maxsize=4)
def f(x):
    return x * x

for i in [1,2,3,2,1,4,5,2]:
    _ = f(i)
print(f.cache_info())
# CacheInfo(hits=3, misses=5, maxsize=4, currsize=4)
```

* **hits**: respuestas servidas desde caché.
* **misses**: cómputos “reales”.
* **currsize**: cuántas entradas hay ahora.

---

## 🧭 Modelo mental (cómo aprovecharlo)

1. **Pureza** ✨: la función debe ser “casi pura” (mismos inputs → mismo output, sin efectos colaterales relevantes).
2. **Estado como argumentos** 🧱: todo lo que defina el subproblema debe ir en los parámetros (nada “oculto” en variables globales mutables).
3. **Hashabilidad** 🔑: args/kwargs **hashables** (tuplas, ints, strings, `frozenset`, objetos `frozen=True`), no listas o dicts.
4. **Dominio acotado** 📦: en DP top-down, el número de estados distintos debe ser razonable (evita caches gigantes).
5. **Límite de memoria** 🧮: si el dominio puede ser grande o la función vive mucho (p. ej. en un servicio), usa `maxsize` finito.

---

## 🧩 Patrones típicos en Programación Dinámica

### 1) DP top-down “clásica”

```python
from functools import lru_cache

def lcs_len(a: str, b: str) -> int:
    n, m = len(a), len(b)

    @lru_cache(maxsize=None)
    def dp(i: int, j: int) -> int:
        if i == n or j == m:
            return 0
        if a[i] == b[j]:
            return 1 + dp(i+1, j+1)
        return max(dp(i+1, j), dp(i, j+1))
    return dp(0, 0)
```

* **Clave**: `(i, j)` (hashable).
* **Estados**: `O(n·m)` → cada uno se calcula **una vez**.

### 2) Cuando tienes estructuras mutables en el estado

```python
from functools import lru_cache
from typing import Tuple

def count_paths(blocked: set[Tuple[int,int]], n: int, m: int) -> int:
    blocked_f = frozenset(blocked)  # congelar para hash

    @lru_cache(maxsize=None)
    def dp(i: int, j: int) -> int:
        if (i, j) in blocked_f or i >= n or j >= m:
            return 0
        if i == n-1 and j == m-1:
            return 1
        return dp(i+1, j) + dp(i, j+1)
    return dp(0, 0)
```

* Convierte **mutables → inmutables** (`set`→`frozenset`, `list`→`tuple`).

---

## ⚠️ Errores comunes (y soluciones)

* **Argumentos no hashables** ❌

  * *Síntoma:* `TypeError: unhashable type: 'list'`.
  * ✅ Solución: convertir a tuplas/frozensets o usar tipos `frozen`.

* **Side effects** (acceder I/O, reloj, globales) ⏱️

  * *Riesgo:* resultados “viejos” servidos desde caché.
  * ✅ Solución: limita la caché a funciones deterministas; si el estado global cambió, llama `cache_clear()`.

* **Fuga de memoria por `maxsize=None`** 🧨

  * En procesos largos o servidores, la caché puede crecer indefinidamente.
  * ✅ Solución: establece `maxsize` finito acorde al patrón de acceso.

* **Aplicarlo a métodos sin pensar en `self`** 🧩

  * `self` entra en la clave. Si creas muchas instancias, **duplicas entradas**.
  * ✅ Opciones:

    * Decorar funciones “libres” que reciban estados explícitos (no `self`), o
    * Usar `@cached_property` si no hay argumentos, o
    * Asegurar que la clase sea hashable y que tiene sentido cachear *por instancia*.

* **Recursión profunda** 🌊

  * `lru_cache` no evita `RecursionError`.
  * ✅ Plantea **tabulation** o reestructura el orden para reducir profundidad.

---

## 🧵 Concurrencia y procesos

* **Thread-safe** 🔒: internamente usa un lock para proteger la estructura; apto para multi-hilo en **el mismo proceso**.
* **Multiproceso** 🧩: la caché **no se comparte** entre procesos (memoria separada).

---

## ⏱️ Complejidad (alto nivel)

* **Búsqueda/actualización**: amortizado **O(1)** por operación (dict + LRU).
* **Memoria**: **O(currsize)** (número de entradas actualmente en la caché).
* **DP Top-Down**: tiempo ≈ **#estados distintos × costo por estado** (gracias a que cada estado se computa una sola vez).

---

## 🧰 Trucos prácticos

* **Medir**:

  ```python
  res = lcs_len("abc", "ac")
  print(lcs_len.__wrapped__.cache_info())  # o lcs_len.cache_info() según dónde definas el decorador
  ```
* **Reset por cambio de parámetros globales**:

  ```python
  lcs_len.cache_clear()
  ```
* **Separar por tipo**:

  ```python
  @lru_cache(maxsize=1024, typed=True)
  def f(x): ...
  ```
* **Clave custom** (cuando hay muchos args): empaqueta tú mismo una tupla mínima y llama a una función interna cacheada con esa tupla.

---

## 🧪 Mini-demo didáctica

```python
from functools import lru_cache

calls = 0

@lru_cache(maxsize=None)
def fib(n: int) -> int:
    global calls
    calls += 1
    return n if n < 2 else fib(n-1) + fib(n-2)

print(fib(35))
print("Llamadas reales:", calls)           # ~36
print(fib.cache_info())                    # hits altos, misses ~n+1
```

* Sin caché, `fib(35)` hace \~14 millones de llamadas; con `lru_cache`, **solo \~n**.

---

## 🧩 Cuándo **sí** vs **no** usar `lru_cache`

**Úsalo si…**

* La función es **determinista** y el dominio de entradas **se repite** (DP top-down, parsers, combinatoria).
* Quieres acelerar computos costosos reutilizados (p. ej. parseo de tokens, factorizaciones, distancias).

**Evítalo si…**

* La función depende de **tiempo/reloj**, I/O o fuentes externas que cambian.
* El dominio de entradas **siempre es nuevo** (no hay reutilización) o es **enorme** y no quieres ocupar memoria.

---

## 🏁 Resumen operativo

* **Decoras** con `@lru_cache(maxsize=..., typed=...)`.
* Verificas que **argumentos** sean **hashables**.
* **Diseñas el estado** explícito (clave mínima suficiente).
* Mides con `cache_info()` y limpias con `cache_clear()` cuando cambie el contexto.
* En DP: piensa **estados, casos base y transición**; el resto lo hace la caché.

---

# 🧭 Guía: ¿Cuándo aplicar Programación Dinámica con Memoization?

## 1. Propiedades necesarias

Para que un problema se resuelva bien con PD (memoization o tabulation), debe cumplir al menos **dos condiciones fundamentales**:

1. **Subestructura óptima** ⚙️

   * La solución óptima al problema puede construirse a partir de soluciones óptimas a subproblemas.
   * Ejemplo: el camino más corto de A a C pasando por B es la suma del camino más corto de A→B y B→C.

2. **Subproblemas superpuestos** 🔁

   * El mismo subproblema aparece una y otra vez en el árbol de recursión.
   * Ejemplo: en Fibonacci, `fib(3)` se calcula múltiples veces si no hay memoization.

---

## 2. Modelo de pensamiento: checklist mental 🧠

### Paso 1. ¿Puedo **descomponer** el problema?

* ¿El problema grande se puede reducir en versiones más pequeñas del mismo tipo?
* Ejemplo: subir `n` escalones → subir `n-1` y `n-2`.

👉 Si no hay una descomposición natural, PD probablemente no aplica.

---

### Paso 2. ¿Se repiten los subproblemas?

* Dibuja (o imagina) el árbol de recursión de una versión naive.
* Marca si los mismos estados aparecen varias veces.
* Ejemplo: en LCS, el estado `(i,j)` aparece en muchos caminos del árbol recursivo.

👉 Si no se repiten, mejor usar **divide & conquer** (ej: merge sort) que PD.

---

### Paso 3. ¿Cuál es el **estado mínimo** necesario?

* Define qué parámetros identifican de manera única a un subproblema.
* Estos parámetros deben ser **hashables** (enteros, strings, tuplas, etc.).
* Ejemplo:

  * Climbing stairs → `n`.
  * LCS → `(i,j)`.
  * Knapsack → `(i, capacidad)`.

👉 Regla: si dos llamadas con los mismos parámetros representan exactamente el mismo problema, ya tienes el estado correcto.

---

### Paso 4. ¿Existen **casos base** claros?

* ¿Puedes definir condiciones que terminen la recursión?
* Ejemplo:

  * Fib: `n=0` o `n=1`.
  * Caminos en grilla: cuando llegas a la meta o sales de la grilla.

👉 Si no hay condiciones naturales para cortar, no es buen candidato.

---

### Paso 5. ¿Cómo es la **transición**?

* Pregunta: ¿cómo combino resultados de subproblemas para obtener la solución?
* Patrones típicos:

  * **Suma** de opciones (conteo).
  * **Min/Max** de opciones (optimización).
  * **Condicional** (si se cumple algo, avanza en diagonal; si no, prueba ramas).

---

### Paso 6. ¿Cuál es la **complejidad de estados**?

* Calcula cuántos estados distintos existen = producto de rangos de parámetros.
* Estima el costo por estado (O(1), O(n), etc.).
* Complejidad total = #estados × costo por estado.
* Ejemplo:

  * LCS: `O(n·m)` estados, O(1) cada uno → O(n·m).
  * Coin Change: `O(n·amount)`.

👉 Si el # de estados es manejable, aplica PD. Si explota (ej: estados exponenciales), no conviene.

---

### Paso 7. ¿Memoization o Tabulation?

* **Memoization (top-down)**:

  * Si la recursión es natural.
  * Si no vas a explorar todos los estados.
* **Tabulation (bottom-up)**:

  * Si quieres evitar call stack profundo.
  * Si es más claro construir la tabla iterativamente.

---

# 📚 Mini ejemplo de aplicación del modelo

**Problema:** "Número de formas de decodificar un string numérico (‘1’→A,…,‘26’→Z)."

1. **Descomposición**: para decodificar `s[i:]`, puedo tomar 1 dígito (`dp(i+1)`) o 2 dígitos (`dp(i+2)`).
2. **Subproblemas superpuestos**: el mismo índice `i` aparece varias veces.
3. **Estado mínimo**: basta con el índice `i`.
4. **Casos base**: `i==len(s) ⇒ 1`, si `s[i]=='0' ⇒ 0`.
5. **Transición**: suma de opciones válidas.
6. **Complejidad**: O(n) estados, cada uno O(1) → O(n).
7. **Elección**: memoization porque es muy natural definirlo recursivo.

---

# ✅ Resumen para estudiantes

**Un problema es candidato a PD con memoization si:**

1. Se puede descomponer en subproblemas más pequeños.
2. Esos subproblemas se repiten muchas veces.
3. Se puede definir un estado único y casos base claros.
4. Existe una regla (transición) para combinar subsoluciones.
5. El número total de estados es manejable.

👉 **Memoization = un diccionario (hashmap) donde cada clave es un subproblema y el valor es su solución.**

---
